In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [15]:
N = 100
K = 10000
ua = 0
ub = 4
T = 1.0
h = (ub - ua) / N
tau = T / K
a = 16 / np.pi


def f(x, t):
    return np.exp(-np.pi * t) * np.sin(np.pi * x / 4)

def u0(x):
    return np.sin((np.pi * x) / 4)

def u_a(t):
    return 0

def u_b(t):
    return 0

In [16]:
def explicit_scheme(a, tau, f, N, K, ua, ub):
    r = a * tau / h**2
    print(f"r = {r}")


    x = np.linspace(ua, ub, N+1)
    t = np.linspace(0, T, K+1)
    u = np.zeros((K+1, N+1))


    u[0, :] = u0(x)
    u[:, 0] = u_a(t)
    u[:, -1] = u_b(t)

    for k in range(0, K):
        for i in range(1, N):
            u[k+1, i] = u[k, i] + r * (u[k, i+1] - 2*u[k, i] + u[k, i-1])

    return x, t, u

In [17]:
x = np.linspace(ua, ub, N+1)
t = np.linspace(0, T, K+1)
u_exact = np.zeros((K+1, N+1))

for k in range(K+1):
  u_exact[k, :] = f(x, t[k])

In [18]:
x, t, u = explicit_scheme(a, tau, f, N, K, ua, ub)


fig = plt.figure(figsize=(18, 6))


# ax1 = fig.add_subplot(131, projection='3d')
# X, T = np.meshgrid(x, t)
# surf_num = ax1.plot_surface(X, T, u, cmap='viridis')
# ax1.set_title('Численное решение')
# ax1.set_xlabel('x')
# ax1.set_ylabel('t')
# ax1.set_zlabel('u')


# ax2 = fig.add_subplot(132, projection='3d')
# surf_exact = ax2.plot_surface(X, T, u_exact, cmap='viridis')
# ax2.set_title('Точное решение')
# ax2.set_xlabel('x')
# ax2.set_ylabel('t')
# ax2.set_zlabel('u')


# ax3 = fig.add_subplot(133, projection='3d')
# surf_diff = ax3.plot_surface(X, T, np.abs(u - u_exact), cmap='inferno')
# ax3.set_title('Ошибка')
# ax3.set_xlabel('x')
# ax3.set_ylabel('t')
# ax3.set_zlabel('Ошибка')

# plt.show()
print(np.max(np.abs(u - u_exact)))
print(h)

r = 0.31830988618379064
2.753205917960244e-05
0.04


<Figure size 1800x600 with 0 Axes>

In [19]:
def implicit_scheme(a, tau, N, K, ua, ub):
    r = a * tau / (2* h**2)

    x = np.linspace(ua, ub, N+1)
    t = np.linspace(0, T, K+1)
    u = np.zeros((K+1, N+1))

    u[0, :] = u0(x)


    u[:, 0] = u_a(t)
    u[:, -1] = u_b(t)


    A = np.zeros(N-1)
    B = np.zeros(N-1)
    C = np.zeros(N-1)

    F = np.zeros(N-1)

    for k in range(0, K):
        for i in range(1, N):
            F[i-1] = (1 - 2 * r) * u[k, i] + r * (u[k, i+1] + u[k, i-1])

        F[0] += r * u[k+1, 0]
        F[-1] += r * u[k+1, -1]

        A[:] = -r
        B[:] = 1 + 2 * r
        C[:] = -r

        U = np.zeros(N-1)
        alpha = np.zeros(N-1)
        beta = np.zeros(N-1)

        alpha[0] = -C[0] / B[0]
        beta[0] = F[0] / B[0]

        for i in range(1, N-1):
            denom = B[i] + A[i] * alpha[i-1]
            alpha[i] = -C[i] / denom
            beta[i] = (F[i] - A[i] * beta[i-1]) / denom

        U[-1] = beta[-1]
        for i in range(N-3, -1, -1):
            U[i] = alpha[i] * U[i+1] + beta[i]

        u[k+1, 1:N] = U

    return x, t, u

In [20]:
x, t, u = implicit_scheme(a, tau, N, K, ua, ub)


fig = plt.figure(figsize=(18, 6))


# ax1 = fig.add_subplot(131, projection='3d')
# X, T = np.meshgrid(x, t)
# surf_num = ax1.plot_surface(X, T, u, cmap='viridis')
# ax1.set_title('Численное решение (неявная схема)')
# ax1.set_xlabel('x')
# ax1.set_ylabel('t')
# ax1.set_zlabel('u')

# ax2 = fig.add_subplot(132, projection='3d')
# surf_exact = ax2.plot_surface(X, T, u_exact, cmap='viridis')
# ax2.set_title('Точное решение')
# ax2.set_xlabel('x')
# ax2.set_ylabel('t')
# ax2.set_zlabel('u')

# ax3 = fig.add_subplot(133, projection='3d')
# surf_diff = ax3.plot_surface(X, T, np.abs(u - u_exact), cmap='inferno')
# ax3.set_title('Ошибка')
# ax3.set_xlabel('x')
# ax3.set_ylabel('t')
# ax3.set_zlabel('Ошибка')

# plt.show()
print(f'ОШИБКА {np.max(np.abs(u - u_exact))}')

ОШИБКА 3.025409488721653e-05


<Figure size 1800x600 with 0 Axes>

## solve

In [ ]:
def implicit_scheme(a, tau, N, K, ua, ub):
    r = a * tau / (2 * h**2)
    print(r)
    x = np.linspace(ua, ub, N+1)
    t = np.linspace(0, T, K+1)
    u = np.zeros((K+1, N+1))

    u[0, :] = u0(x)


    u[:, 0] = u_a(t)
    u[:, -1] = u_b(t)

    # Коэффициенты для трехдиагональной матрицы
    A = np.zeros((N-1, N-1))
    #B = np.zeros(N-1)
    #C = np.zeros(N-1)
    F = np.zeros(N-1)

    for k in range(0, K):
        for i in range(1, N):
            F[i-1] = u[k, i] + r * (u[k, i+1] - 2 * u[k, i] + u[k, i-1])

        # Граничные условия в F
        F[0] += r * u[k+1, 0]
        F[-1] += r * u[k+1, -1]

        # Матрица системы (диагональные коэффициенты)
        for i in range(N-1):
          if i == 0:
              A[i, i] = 1 + 2 * r
              A[i, i+1] = -r
          elif i == N-2:
              A[i, i-1] = -r
              A[i, i] = 1 + 2 * r
          else:
              A[i, i-1] = -r
              A[i, i] = 1 + 2 * r
              A[i, i+1] = -r

        # Решаем трехдиагональную систему методом прогонки
        #B[:] = 1 + 2 * r
        #C[:] = -r

        # Решаем трехдиагональную систему методом прогонки
        U = np.linalg.solve(A, F)

        # Заполняем слой
        u[k+1, 1:N] = U

    return x, t, u